In [ ]:
%matplotlib qt5

from pathlib import Path
import pickle

from orix import io, plot
from orix.vector import Vector3d

import hyperspy.api as hs #General hyperspy package
import pyxem as pxm #Electron diffraction tools based on hyperspy
import numpy as np #General numerical and matrix support
import matplotlib.pyplot as plt #Plotting tools
import matplotlib.colors as mcolors #Some plotting color tools
from matplotlib.cm import ScalarMappable
import diffpy #Electron diffraction tools
import requests

#Import indexation and plotting tools
from diffsims.generators.rotation_list_generators import get_beam_directions_grid
from diffsims.libraries.structure_library import StructureLibrary
from diffsims.generators.diffraction_generator import DiffractionGenerator
from diffsims.generators.library_generator import DiffractionLibraryGenerator
from pyxem.utils import indexation_utils as iutls
from pyxem.utils import plotting_utils as putls
from pyxem.utils import polar_transform_utils as ptutls
from pyxem.utils import expt_utils as eutls

#Import orientation handling tools
from orix.quaternion import Rotation, symmetry, Orientation
from orix.vector.vector3d import Vector3d
from orix.projections import StereographicProjection
from orix import plot

import matplotlib
matplotlib.rcParams["backend"] = "Agg"
from orix.crystal_map.crystal_map import CrystalMap

# One notebook for all the processes needed to template match centered data

## Template matching

In [ ]:
def change_step_size(xmap, pixel_size, scan_unit):
    """Change the step size of an orix CrystalMap
    """
    x = xmap.x * pixel_size
    y = xmap.y * pixel_size
    rot = xmap.rotations
    phaseid = xmap.phase_id
    prop = xmap.prop
    is_in_data = xmap.is_in_data
    phaselist = xmap.phases
    new_xmap = CrystalMap(rotations = rot,
                        phase_id = phaseid,
                        x = x,
                        y = y,
                        prop = prop,
                        scan_unit = scan_unit,
                        is_in_data = is_in_data,
                        phase_list=phaselist)
    return new_xmap

In [ ]:
# Template matching and plotting
def tm_and_plot(libpath,datapath,name): 
    rot_z = 150. 
    pixel_size = 20.372
    scan_unit = "nm"
    # load data
    with open(libpath, 'rb') as fp: 
        library = pickle.load(fp)
    processed_data = hs.load(str(datapath), lazy=True)
    cs = symmetry.Oh
    # Correct scale
    scale = 0.009451434347767504
    processed_data.set_diffraction_calibration(scale)
    processed_data.set_scan_calibration(scale)

    frac_keep = 1 
    result, phasedict = iutls.index_dataset_with_template_rotation(processed_data.inav[::1],
                                                        library,
                                                    phases = ["LNMO"],  # if we have multiple phases we can also specify which ones we want to consider. If it's not specified, all phases are used.
                                                    n_best = 1, 
                                                    frac_keep = frac_keep,
                                                    n_keep = None,
                                                    delta_r = 1,
                                                    delta_theta = 1,
                                                    max_r = np.min(processed_data.axes_manager.signal_shape)//2,
                                                    intensity_transform_function=lambda x: x**0.09,
                                                    normalize_images = True,
                                                    normalize_templates=True,
                                                    )
    
    # Correct for scan rotation
    result["orientation"][:,:,:,0] = result["orientation"][:,:,:,0] + rot_z
    # Set scale
    
    xmap = iutls.results_dict_to_crystal_map(result, phasedict, diffraction_library=library)
    xmap.phases[0].space_group = 227 

    xmap.correlation = np.nan_to_num(xmap.correlation) # If any correlation scores are NaN
    xmap = change_step_size(xmap, pixel_size, scan_unit)

    # Save as h5 and ang format
    io.save(f"./{name}.h5", xmap) 
    io.save(f"./{name}.ang",xmap)

    # Make crystal map
    ipf_key = plot.IPFColorKeyTSL(xmap.phases[0].point_group, direction=Vector3d.xvector())
    rgb = ipf_key.orientation2color(xmap.orientations)
    fig = xmap.plot(rgb, remove_padding=True, return_figure=True)


## Run template matching for dataset

In [ ]:
tm_and_plot("./library.pkl",'./example_sample_dataset.zspy','xmap')